In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline
# import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D


/Applications/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# loading facial expression dataset from Kaggle

In [9]:
testData = 'datasetPictures/test/test'
trainData = 'datasetPictures/train/train'
# def loadDataset(directory): 
#     imagePaths = []
#     labels = []

#     for label in os.listdir(directory):
#         for filename in os.listdir(directory+label): 
#             imagePath = os.path.join(directory, label, filename)
#             imagePaths.append(imagePath) #input: image
#             labels.append(label) #output: label

#         print(label, "Completed")
#     return imagePaths, labels

imagesTrain = []
labelsTrain = []
print("Loading Train Dataset")
for label in os.listdir(trainData):
    if not label.startswith('.DS'):
        for picture in os.listdir(trainData+'/'+label):
            picPath = os.path.join(trainData, label, picture)
            imagesTrain.append(picPath) #input: image
            labelsTrain.append(label) #output: label
    print(label, "Completed")

imagesTest = []
labelsTest = []
print("Loading Test Dataset")

for label in os.listdir(testData):
    if not label.startswith('.DS'):
        for picture in os.listdir(testData+'/'+label):
            picPath = os.path.join(testData, label, picture)
            imagesTest.append(picPath) #input: image
            labelsTest.append(label) #output: label
    print(label, "Completed")



Loading Train Dataset
happy Completed
.DS_Store Completed
sad Completed
fear Completed
surprise Completed
neutral Completed
angry Completed
disgust Completed
Loading Test Dataset
happy Completed
.DS_Store Completed
sad Completed
fear Completed
surprise Completed
neutral Completed
angry Completed
disgust Completed


# creating dataframe

In [12]:
# training
train = pd.DataFrame()
train ['image'] = imagesTrain
train['label'] = labelsTrain
# shuffle the dataset
train = train.sample(frac = 1).reset_index(drop = True) 
train.head()

,image,label
0,datasetPictures/train/train/sad/Training_70651...,sad
1,datasetPictures/train/train/happy/Training_647...,happy
2,datasetPictures/train/train/angry/Training_943...,angry
3,datasetPictures/train/train/surprise/Training_...,surprise
4,datasetPictures/train/train/surprise/Training_...,surprise


In [15]:
#testing
test = pd.DataFrame()
test ['image'] = imagesTest
test['label'] = labelsTest
test.head()

,image,label
0,datasetPictures/test/test/happy/PrivateTest_45...,happy
1,datasetPictures/test/test/happy/PrivateTest_61...,happy
2,datasetPictures/test/test/happy/PublicTest_536...,happy
3,datasetPictures/test/test/happy/PrivateTest_52...,happy
4,datasetPictures/test/test/happy/PublicTest_411...,happy
